In [1]:
# Somehow figure out what to send to LCO, and preferably edit submit files manually

In [98]:
#%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import marshaltools
import re
from Observatory import Observatory
from utils import get_config, plot_visibility, prepare_snifs_schedule
from astropy.time import Time
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.sdss import SDSS
from io import StringIO
import json, copy
import requests

In [78]:
# Get the lco token
apitoken = { "token": "bf81e4a699d2f65529753cb65f8cce176b47d897" }

In [3]:
apl = marshaltools.ProgramList('AMPEL Test',load_candidates=False)
cpl = marshaltools.ProgramList('Cosmology',load_candidates=False)

INFO:marshaltools.ProgramList:Initialized ProgramList for program AMPEL Test (ID 4)
INFO:marshaltools.ProgramList:Loaded 523 saved sources for program AMPEL Test.
INFO:marshaltools.ProgramList:Initialized ProgramList for program Cosmology (ID 2)
INFO:marshaltools.ProgramList:Loaded 496 saved sources for program Cosmology.


In [43]:
# Look at which of the live SNe can be triggered
# Stupid manual list
trigger = 'ZTF18aceyjfx ZTF18acbwghc ZTF18acbzvuu ZTF18acbzoxo ZTF18acepbxq ZTF18acbycjw ZTF18accnbgw ZTF18acbzvrn ZTF18acbxsec ZTF18acbvgqw ZTF18acbvugp ZTF18accvmgs ZTF18accndxn ZTF18acejgrz ZTF18acemhyb ZTF18acebefc ZTF18acchhpe ZTF18acbvkwl'.split(' ')
lcodone = 'ZTF18acbxsec ZTF18acbvugp'.split('\ ')

In [36]:
# Check visibility
observatories ={
    'uh88': Observatory('uh88', 19.8231, -155.47, 4205, sun_alt_th=-17, logger=None),
}
obs = 'uh88'
date = '2018-11-14'
tshift = {'ntt': -0.25, 'uh88': 0}  # In units of days
trange = [Time(Time(date).jd+tshift[obs],format='jd'), Time(Time(date).jd+7.+tshift[obs],format='jd')]

INFO:root:Initialized Observatory uh88 at position (lon 19.82 deg, lat -155.47 deg, alt: 4205.0 m)


In [99]:
aentries = {}
for snname in trigger:
    if snname in lcodone:
        print('Already done once, skip until request for more data')
        continue
    ra = apl.sources[snname]['ra']
    dec = apl.sources[snname]['dec']
    print('Calculating visibility of source %s (ra: %f, dec: %f) with %s.'%(snname, ra, dec, obs))
    obs_w = observatories[obs].compute_visibility(ra, dec, trange,airmass_th=1.6,dt_min=30)
    amasses = obs_w['airmass']
    # If we have more than five entries in the good airmass range, submit
    aentries[snname] = len(amasses)
    print('Visibility : %s'%(len(amasses)))


INFO:root:computing visibility of source at (ra: 187.092185, dec: 62.383324) from observatory uh88
INFO:root:using visibility constraints:
					-Time resolution: 30.00 min
					-Airmass limit: 1.60
					-Sun altitude: -17.00 deg
					-Moon distance: 30.00 deg


Calculating visibility of source ZTF18aceyjfx (ra: 187.092185, dec: 62.383324) with uh88.


INFO:root:computed dark times (sun_alt: -17.00) between 2018-11-14 00:00:00.000 and 2018-11-27 23:30:00.000. Total of 147.00 hours of dark
INFO:root:source is visible for a total of 0.000 hours. Took 1.28e+00 sec
INFO:root:computing visibility of source at (ra: 169.484828, dec: 27.591036) from observatory uh88
INFO:root:using visibility constraints:
					-Time resolution: 30.00 min
					-Airmass limit: 1.60
					-Sun altitude: -17.00 deg
					-Moon distance: 30.00 deg


Visibility : 0
Calculating visibility of source ZTF18acbwghc (ra: 169.484828, dec: 27.591036) with uh88.


INFO:root:computed dark times (sun_alt: -17.00) between 2018-11-14 00:00:00.000 and 2018-11-27 23:30:00.000. Total of 147.00 hours of dark
INFO:root:source is visible for a total of 21.000 hours. Took 1.39e+00 sec
INFO:root:computing visibility of source at (ra: 169.072218, dec: 47.488951) from observatory uh88
INFO:root:using visibility constraints:
					-Time resolution: 30.00 min
					-Airmass limit: 1.60
					-Sun altitude: -17.00 deg
					-Moon distance: 30.00 deg


...

Visibility : 148
Calculating visibility of source ZTF18acchhpe (ra: 357.879922, dec: 28.600345) with uh88.


INFO:root:computed dark times (sun_alt: -17.00) between 2018-11-14 00:00:00.000 and 2018-11-27 23:30:00.000. Total of 147.00 hours of dark
INFO:root:source is visible for a total of 76.000 hours. Took 1.59e+00 sec
INFO:root:computing visibility of source at (ra: 70.815493, dec: -8.605053) from observatory uh88
INFO:root:using visibility constraints:
					-Time resolution: 30.00 min
					-Airmass limit: 1.60
					-Sun altitude: -17.00 deg
					-Moon distance: 30.00 deg


Visibility : 152
Calculating visibility of source ZTF18acbvkwl (ra: 70.815493, dec: -8.605053) with uh88.


INFO:root:computed dark times (sun_alt: -17.00) between 2018-11-14 00:00:00.000 and 2018-11-27 23:30:00.000. Total of 147.00 hours of dark
INFO:root:Too close the the moon.
INFO:root:source is visible for a total of 74.000 hours. Took 1.60e+00 sec


Visibility : 148


In [52]:
# Load LCO template
with open('lcoapi.json','r') as fh:
    api = json.load(fh)

In [93]:
lcoout = {}
count = 0
for snname, vis in aentries.items():
    if vis < 5:
        print('%s not visible, skip'%(snname))
        continue
    
    # No need to resubmit
    count +=1
    if count <= 3:
        continue
    
    
    snapi = copy.deepcopy(api)

    
    # target
    snapi['requests'][0]['target']['name'] = snname
    snapi['requests'][0]['target']['ra'] = apl.sources[snname]['ra']
    snapi['requests'][0]['target']['dec'] = apl.sources[snname]['dec']
    
    # windows we leave as default (could change default window)
    # location is also default (2m telescope)
    snapi['requests'][0]['constraints']['max_airmass'] = 1.6
    
    # We could leave the moleculres (obs) as default, or change slidwith
    for mol in snapi['requests'][0]['molecules']:
        mol['spectra_slit'] = 'slit_1.6as'
    
#    print(snapi)
    
    apiresponse = requests.post(
#        'https://observe.lco.global/api/userrequests/validate/',
        'https://observe.lco.global/api/userrequests/',
        headers={'Authorization':'Token {}'.format(apitoken['token'])},
        json=snapi    )
    
    
    # Make sure this api call was successful
    try:
        apiresponse.raise_for_status()
    except requests.exceptions.HTTPError as exc:
        print('Request failed: {}'.format(apiresponse.content))
        raise exc
    
    
    print(apiresponse.json())
    lcoout[snname] = apiresponse.json()
    
#    with open('lco_%s.json'%(snname),'w') as fh:
#        json.dump(snapi,fh)
        
        

ZTF18aceyjfx not visible, skip
{'id': 719962, 'requests': [{'id': 1670919, 'location': {'telescope_class': '2m0'}, 'constraints': {'max_airmass': 1.6, 'min_lunar_distance': 30.0, 'max_lunar_phase': None, 'max_seeing': None, 'min_transparency': None}, 'target': {'type': 'SIDEREAL', 'name': 'ZTF18acepbxq', 'ra': 158.3969031, 'dec': 15.0994233, 'proper_motion_ra': 0.0, 'proper_motion_dec': 0.0, 'parallax': 0.0, 'radvel': 0.0, 'coordinate_system': 'ICRS', 'equinox': 'J2000', 'epoch': 2000.0, 'acquire_mode': None, 'rot_mode': 'VFLOAT', 'rot_angle': 0.0, 'vmag': None}, 'molecules': [{'type': 'LAMP_FLAT', 'args': '', 'priority': 1, 'ag_name': '', 'ag_mode': 'OPTIONAL', 'ag_filter': '', 'ag_exp_time': 10.0, 'ag_strategy': '', 'instrument_name': '2M0-FLOYDS-SCICAM', 'filter': '', 'readout_mode': '', 'spectra_lamp': '', 'spectra_slit': 'slit_1.6as', 'acquire_mode': 'WCS', 'acquire_radius_arcsec': 0.0, 'acquire_strategy': '', 'expmeter_mode': 'OFF', 'expmeter_snr': None, 'exposure_time': 60.0, 'e

{'id': 719965, 'requests': [{'id': 1670922, 'location': {'telescope_class': '2m0'}, 'constraints': {'max_airmass': 1.6, 'min_lunar_distance': 30.0, 'max_lunar_phase': None, 'max_seeing': None, 'min_transparency': None}, 'target': {'type': 'SIDEREAL', 'name': 'ZTF18acbzvrn', 'ra': 166.5545491, 'dec': 50.2612422, 'proper_motion_ra': 0.0, 'proper_motion_dec': 0.0, 'parallax': 0.0, 'radvel': 0.0, 'coordinate_system': 'ICRS', 'equinox': 'J2000', 'epoch': 2000.0, 'acquire_mode': None, 'rot_mode': 'VFLOAT', 'rot_angle': 0.0, 'vmag': None}, 'molecules': [{'type': 'LAMP_FLAT', 'args': '', 'priority': 1, 'ag_name': '', 'ag_mode': 'OPTIONAL', 'ag_filter': '', 'ag_exp_time': 10.0, 'ag_strategy': '', 'instrument_name': '2M0-FLOYDS-SCICAM', 'filter': '', 'readout_mode': '', 'spectra_lamp': '', 'spectra_slit': 'slit_1.6as', 'acquire_mode': 'WCS', 'acquire_radius_arcsec': 0.0, 'acquire_strategy': '', 'expmeter_mode': 'OFF', 'expmeter_snr': None, 'exposure_time': 60.0, 'exposure_count': 2, 'bin_x': 1, 

{'id': 719968, 'requests': [{'id': 1670925, 'location': {'telescope_class': '2m0'}, 'constraints': {'max_airmass': 1.6, 'min_lunar_distance': 30.0, 'max_lunar_phase': None, 'max_seeing': None, 'min_transparency': None}, 'target': {'type': 'SIDEREAL', 'name': 'ZTF18acbvugp', 'ra': 95.1821106, 'dec': 59.3426393, 'proper_motion_ra': 0.0, 'proper_motion_dec': 0.0, 'parallax': 0.0, 'radvel': 0.0, 'coordinate_system': 'ICRS', 'equinox': 'J2000', 'epoch': 2000.0, 'acquire_mode': None, 'rot_mode': 'VFLOAT', 'rot_angle': 0.0, 'vmag': None}, 'molecules': [{'type': 'LAMP_FLAT', 'args': '', 'priority': 1, 'ag_name': '', 'ag_mode': 'OPTIONAL', 'ag_filter': '', 'ag_exp_time': 10.0, 'ag_strategy': '', 'instrument_name': '2M0-FLOYDS-SCICAM', 'filter': '', 'readout_mode': '', 'spectra_lamp': '', 'spectra_slit': 'slit_1.6as', 'acquire_mode': 'WCS', 'acquire_radius_arcsec': 0.0, 'acquire_strategy': '', 'expmeter_mode': 'OFF', 'expmeter_snr': None, 'exposure_time': 60.0, 'exposure_count': 2, 'bin_x': 1, '

{'id': 719971, 'requests': [{'id': 1670928, 'location': {'telescope_class': '2m0'}, 'constraints': {'max_airmass': 1.6, 'min_lunar_distance': 30.0, 'max_lunar_phase': None, 'max_seeing': None, 'min_transparency': None}, 'target': {'type': 'SIDEREAL', 'name': 'ZTF18acemhyb', 'ra': 119.1422834, 'dec': 30.1766162, 'proper_motion_ra': 0.0, 'proper_motion_dec': 0.0, 'parallax': 0.0, 'radvel': 0.0, 'coordinate_system': 'ICRS', 'equinox': 'J2000', 'epoch': 2000.0, 'acquire_mode': None, 'rot_mode': 'VFLOAT', 'rot_angle': 0.0, 'vmag': None}, 'molecules': [{'type': 'LAMP_FLAT', 'args': '', 'priority': 1, 'ag_name': '', 'ag_mode': 'OPTIONAL', 'ag_filter': '', 'ag_exp_time': 10.0, 'ag_strategy': '', 'instrument_name': '2M0-FLOYDS-SCICAM', 'filter': '', 'readout_mode': '', 'spectra_lamp': '', 'spectra_slit': 'slit_1.6as', 'acquire_mode': 'WCS', 'acquire_radius_arcsec': 0.0, 'acquire_strategy': '', 'expmeter_mode': 'OFF', 'expmeter_snr': None, 'exposure_time': 60.0, 'exposure_count': 2, 'bin_x': 1, 

{'id': 719974, 'requests': [{'id': 1670931, 'location': {'telescope_class': '2m0'}, 'constraints': {'max_airmass': 1.6, 'min_lunar_distance': 30.0, 'max_lunar_phase': None, 'max_seeing': None, 'min_transparency': None}, 'target': {'type': 'SIDEREAL', 'name': 'ZTF18acbvkwl', 'ra': 70.8154927, 'dec': -8.6050531, 'proper_motion_ra': 0.0, 'proper_motion_dec': 0.0, 'parallax': 0.0, 'radvel': 0.0, 'coordinate_system': 'ICRS', 'equinox': 'J2000', 'epoch': 2000.0, 'acquire_mode': None, 'rot_mode': 'VFLOAT', 'rot_angle': 0.0, 'vmag': None}, 'molecules': [{'type': 'LAMP_FLAT', 'args': '', 'priority': 1, 'ag_name': '', 'ag_mode': 'OPTIONAL', 'ag_filter': '', 'ag_exp_time': 10.0, 'ag_strategy': '', 'instrument_name': '2M0-FLOYDS-SCICAM', 'filter': '', 'readout_mode': '', 'spectra_lamp': '', 'spectra_slit': 'slit_1.6as', 'acquire_mode': 'WCS', 'acquire_radius_arcsec': 0.0, 'acquire_strategy': '', 'expmeter_mode': 'OFF', 'expmeter_snr': None, 'exposure_time': 60.0, 'exposure_count': 2, 'bin_x': 1, '

In [95]:
# Now look at cpl SNIa for redshift
obs = 'uh88'
date = '2018-11-14'
tshift = {'ntt': -0.25, 'uh88': 0}  # In units of days
trange = [Time(Time(date).jd+tshift[obs],format='jd'), Time(Time(date).jd+14.+tshift[obs],format='jd')]

In [100]:
for snname, sninfo in cpl.sources.items():
    if sninfo['classification'] is None: continue
    if not re.search('Ia',sninfo['classification']) : continue
        
    ra = cpl.sources[snname]['ra']
    dec = cpl.sources[snname]['dec']
    print('Calculating visibility of source %s (ra: %f, dec: %f) with %s.'%(snname, ra, dec, obs))
    obs_w = observatories[obs].compute_visibility(ra, dec, trange,airmass_th=2.0,dt_min=30)
    amasses = obs_w['airmass']
    if (len(amasses)<10):
        print('Not enough visibility')

    # Check SDSS
    # Explore SDSS query - should do for the full sample
    pos = SkyCoord(ra,dec, frame='icrs', unit="deg")
    response = SDSS.query_crossid_async(pos)
    sdsstab = pd.read_table(StringIO(response.text),sep=',',header=1)
    print(sdsstab)
    continue
    sdsstypes = sdsstab['type']
    if len(sdsstypes) == 0:
        # No SDSS match, continue
        pass
    elif len(sdsstypes) == 1 and cut_sdss_stars:
        if sdsstypes[0]=='STAR':
            msg = "%s: Skipping %s due to photometric SDSS def"%(date,sne[1]["ztf_name"])
            logger.info(msg)
            continue
    else:
        print(sdsstypes)
        print('Multiple SDSS photometric matches...')
    
    
    
    
    
    aentries[snname] = len(amasses)
        
    print('snia')
    print(snname)
    print(sninfo)

INFO:root:computing visibility of source at (ra: 164.223300, dec: 79.278500) from observatory uh88
INFO:root:using visibility constraints:
					-Time resolution: 30.00 min
					-Airmass limit: 2.00
					-Sun altitude: -17.00 deg
					-Moon distance: 30.00 deg


Calculating visibility of source ZTF18aaaoktp (ra: 164.223300, dec: 79.278500) with uh88.


INFO:root:computed dark times (sun_alt: -17.00) between 2018-11-14 00:00:00.000 and 2018-11-27 23:30:00.000. Total of 147.00 hours of dark
INFO:root:source is visible for a total of 9.000 hours. Took 1.40e+00 sec
INFO:root:computing visibility of source at (ra: 166.486468, dec: 37.625131) from observatory uh88
INFO:root:using visibility constraints:
					-Time resolution: 30.00 min
					-Airmass limit: 2.00
					-Sun altitude: -17.00 deg
					-Moon distance: 30.00 deg


Empty DataFrame
Columns: [obj_id, objID, ra, dec, psfMag_u, psfMagerr_u, psfMag_g, psfMagerr_g, psfMag_r, psfMagerr_r, psfMag_i, psfMagerr_i, psfMag_z, psfMagerr_z, obj_id1, type]
Index: []
Calculating visibility of source ZTF18aabasml (ra: 166.486468, dec: 37.625131) with uh88.


INFO:root:computed dark times (sun_alt: -17.00) between 2018-11-14 00:00:00.000 and 2018-11-27 23:30:00.000. Total of 147.00 hours of dark
INFO:root:source is visible for a total of 33.500 hours. Took 1.53e+00 sec
INFO:root:computing visibility of source at (ra: 142.229656, dec: 49.238126) from observatory uh88
INFO:root:using visibility constraints:
					-Time resolution: 30.00 min
					-Airmass limit: 2.00
					-Sun altitude: -17.00 deg
					-Moon distance: 30.00 deg


...

  obj_id                objID          ra        dec  psfMag_u  psfMagerr_u  \
0  obj_0  1237663229605314606  260.812335  77.844025  21.00296      0.12504   

   psfMag_g  psfMagerr_g  psfMag_r  psfMagerr_r  psfMag_i  psfMagerr_i  \
0  20.01774     0.123662  19.36283     1.877244  19.13125     2.345592   

   psfMag_z  psfMagerr_z              obj_id1    type  
0  19.15657     0.061756  1237663229605314606  GALAXY  
Calculating visibility of source ZTF18abnygkb (ra: 328.500035, dec: 8.114686) with uh88.


INFO:root:computed dark times (sun_alt: -17.00) between 2018-11-14 00:00:00.000 and 2018-11-27 23:30:00.000. Total of 147.00 hours of dark
INFO:root:Too close the the moon.
INFO:root:source is visible for a total of 40.500 hours. Took 2.16e+00 sec
INFO:root:computing visibility of source at (ra: 277.889386, dec: 49.816094) from observatory uh88
INFO:root:using visibility constraints:
					-Time resolution: 30.00 min
					-Airmass limit: 2.00
					-Sun altitude: -17.00 deg
					-Moon distance: 30.00 deg


  obj_id                objID          ra       dec  psfMag_u  psfMagerr_u  \
0  obj_0  1237678846647206564  328.500085  8.115032  21.00941     0.068742   

   psfMag_g  psfMagerr_g  psfMag_r  psfMagerr_r  psfMag_i  psfMagerr_i  \
0  18.97706     0.014996  18.20893     0.015605  17.84205      0.02386   

   psfMag_z  psfMagerr_z              obj_id1    type  
0   17.4853     0.021737  1237678846647206564  GALAXY  
Calculating visibility of source ZTF18abpaywm (ra: 277.889386, dec: 49.816094) with uh88.


INFO:root:computed dark times (sun_alt: -17.00) between 2018-11-14 00:00:00.000 and 2018-11-27 23:30:00.000. Total of 147.00 hours of dark
INFO:root:source is visible for a total of 11.500 hours. Took 1.86e+00 sec


ReadTimeout: HTTPConnectionPool(host='skyserver.sdss.org', port=80): Read timed out. (read timeout=60)

In [51]:
api

{'group_id': 'ampelLCO',
 'proposal': 'CON2018B-005',
 'ipp_value': '1.0',
 'operator': 'SINGLE',
 'observation_type': 'NORMAL',
 'requests': [{'acceptability_threshold': 100,
   'target': {'name': 'ZTFSN',
    'type': 'SIDEREAL',
    'ra': '166.554549',
    'dec': '50.261242',
    'proper_motion_ra': 0,
    'proper_motion_dec': 0,
    'epoch': 2000,
    'parallax': 0,
    'rot_mode': 'VFLOAT',
    'rot_angle': 0},
   'molecules': [{'type': 'LAMP_FLAT',
     'instrument_name': '2M0-FLOYDS-SCICAM',
     'exposure_count': '2',
     'bin_x': 1,
     'bin_y': 1,
     'fill_window': False,
     'defocus': 0,
     'ag_mode': 'OPTIONAL',
     'acquire_mode': 'WCS',
     'exposure_time': 60,
     'spectra_slit': 'slit_2.0as'},
    {'type': 'ARC',
     'instrument_name': '2M0-FLOYDS-SCICAM',
     'exposure_count': '2',
     'bin_x': 1,
     'bin_y': 1,
     'fill_window': False,
     'defocus': 0,
     'ag_mode': 'OPTIONAL',
     'acquire_mode': 'WCS',
     'exposure_time': 60,
     'spectra_sl

In [46]:
lcojson = {"group_id":"ampelLCO","proposal":"CON2018B-005","ipp_value":"1.0","operator":"SINGLE","observation_type":"NORMAL","requests":[{"acceptability_threshold":100,"target":{"name":"ZTFSN","type":"SIDEREAL","ra":"166.554549","dec":"50.261242","proper_motion_ra":0,"proper_motion_dec":0,"epoch":2000,"parallax":0,"rot_mode":"VFLOAT","rot_angle":0},"molecules":[{"type":"LAMP_FLAT","instrument_name":"2M0-FLOYDS-SCICAM","exposure_count":"2","bin_x":1,"bin_y":1,"fill_window":false,"defocus":0,"ag_mode":"OPTIONAL","acquire_mode":"WCS","exposure_time":60,"spectra_slit":"slit_2.0as"},{"type":"ARC","instrument_name":"2M0-FLOYDS-SCICAM","exposure_count":"2","bin_x":1,"bin_y":1,"fill_window":false,"defocus":0,"ag_mode":"OPTIONAL","acquire_mode":"WCS","exposure_time":60,"spectra_slit":"slit_2.0as"},{"type":"SPECTRUM","instrument_name":"2M0-FLOYDS-SCICAM","exposure_count":"2","bin_x":1,"bin_y":1,"fill_window":false,"defocus":0,"ag_mode":"ON","acquire_mode":"WCS","exposure_time":"900","spectra_slit":"slit_2.0as"},{"type":"ARC","instrument_name":"2M0-FLOYDS-SCICAM","exposure_count":"2","bin_x":1,"bin_y":1,"fill_window":false,"defocus":0,"ag_mode":"OPTIONAL","acquire_mode":"WCS","exposure_time":60,"spectra_slit":"slit_2.0as"},{"type":"LAMP_FLAT","instrument_name":"2M0-FLOYDS-SCICAM","exposure_count":"2","bin_x":1,"bin_y":1,"fill_window":false,"defocus":0,"ag_mode":"OPTIONAL","acquire_mode":"WCS","exposure_time":60,"spectra_slit":"slit_2.0as"}],"windows":[{"start":"2018-11-14 09:52:13","end":"2018-11-20 09:52:13"}],"location":{"telescope_class":"2m0"},"constraints":{"max_airmass":"1.8","min_lunar_distance":30}}]}

NameError: name 'false' is not defined